In [42]:
seed = 42
proj_id = 'cmat-315920'
root_path  = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

for level in ['cntyvtd']:#'tabblock', 'cntyvtd', 'bg', 'tract', 'cnty']:
    g = Gerry(abbr='TX', level=level,
              election_filters=(
                "office='President' and race='general'",
                "office='USSen' and race='general'",
#                 "office like 'USRep%' and race='general'",
              ),
              refresh_tbl=refresh_tbl, refresh_all=refresh_all
             )
    g.get_data()
#     g.MCMC(1)
self = g

Get crosswalks TX 2010 tabblock  ... tabblock table exists ... success
Get assignments TX 2020 tabblock ... tabblock table exists ... success
Get shapes TX 2020 tabblock      ... tabblock table exists ... success
Get census TX 2010 tabblock      ... tabblock table exists ... success
Get elections TX 2020 tabblock   ... tabblock table exists ... success
Get votes_all TX 2020 tabblock   ... tabblock table exists ... success
Get votes_hl TX 2020 tabblock    ... tabblock table exists ... success
Get combined TX 2020 cntyvtd     ... cntyvtd table exists ... success
Get nodes TX 2020 cntyvtd        ... success
success
Get graph TX 2010 cntyvtd cd     ... creating graph ... get edges first
Get edges TX 2020 cntyvtd        ... cntyvtd table exists ... success
returning to graph ... connecting districts ... 
District   1 component sizes = [273] ... connected
District   2 component sizes = [157] ... connected
District   3 component sizes = [204] ... connected
District   4 component sizes = [339]